# Transformer: causal decoder model

A re-implementation of Andrej Karpahty's GPT from scratch [lecture](https://www.youtube.com/watch?v=kCc8FmEb1nY).

I rewrote it for my own learning. Titles, comments and variables names are focused on conciseness and clarity.

Also, Drake lyrics instead of Shakespeare so we can to accelerate the birth of Drake AI.

## Set up environment

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

## Prepare data

In [2]:
# Read Drake lyrics into memory
with open('../data/drake_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f"{len(text):,}")

770,711


In [3]:
# Preview text
print(text[:1000])

"[Verse]
Put my feelings on ice
Always been a gem
Certified lover boy, somehow still heartless
Heart is only gettin' colder"
"[Verse]
Hands are tied
Someone's in my ear from the other side
Tellin' me that I should pay you no mind
Wanted you to not be with me all night
Wanted you to not stay with me all night
I know, you know, who that person is to me
Doesn't really change things

[Chorus]
I know you're scared of dating, falling for me
Shorty, surely you know me
Right here for you always
You know, I don't ever change
Right here for you always
You know I don't ever change
Right here for you

[Bridge]
In mind you make me want to do things, love you
Like I'm supposed to
You make me want to love you
Like I'm supposed to
You make me want to love you
Like I'm supposed to, remind you
Ayy

[Chorus]
I know you're scared of dating, falling for me
Shorty, by now you know me
Right here for you always
You know, I don't ever change
Right here for you always
You know I don't ever change
Right here for

## Create character vocabulary and embeddings

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$%&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}àáèéëñóú –—‘’“”… 
104


## Set up token encoder and decoder

In [5]:
s2i = { ch:i for i,ch in enumerate(chars) }
i2s = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [s2i[c] for c in s]
decode = lambda l: ''.join([i2s[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[65, 62, 69, 69, 72, 1, 80, 72, 75, 69, 61]
hello world


In [6]:
# Create token stream (`data: Tensor`) from encoded text (tokens)
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([770711]) torch.int64
tensor([ 3, 55, 50, 62, 75, 76, 62, 56,  0, 44, 78, 77,  1, 70, 82,  1, 63, 62,
        62, 69, 66, 71, 64, 76,  1, 72, 71,  1, 66, 60, 62,  0, 29, 69, 80, 58,
        82, 76,  1, 59, 62, 62, 71,  1, 58,  1, 64, 62, 70,  0, 31, 62, 75, 77,
        66, 63, 66, 62, 61,  1, 69, 72, 79, 62, 75,  1, 59, 72, 82, 12,  1, 76,
        72, 70, 62, 65, 72, 80,  1, 76, 77, 66, 69, 69,  1, 65, 62, 58, 75, 77,
        69, 62, 76, 76,  0, 36, 62, 58, 75, 77,  1, 66, 76,  1, 72, 71, 69, 82,
         1, 64, 62, 77, 77, 66, 71,  7,  1, 60, 72, 69, 61, 62, 75,  3,  0,  3,
        55, 50, 62, 75, 76, 62, 56,  0, 36, 58, 71, 61, 76,  1, 58, 75, 62,  1,
        77, 66, 62, 61,  0, 47, 72, 70, 62, 72, 71, 62,  7, 76,  1, 66, 71,  1,
        70, 82,  1, 62, 58, 75,  1, 63, 75, 72, 70,  1, 77, 65, 62,  1, 72, 77,
        65, 62, 75,  1, 76, 66, 61, 62,  0, 48, 62, 69, 69, 66, 71,  7,  1, 70,
        62,  1, 77, 65, 58, 77,  1, 37,  1, 76, 65, 72, 78, 69, 61,  1, 73, 58,
       

## Set up hyperparameters

In [7]:
block_size = 8  # max context length for predictions
batch_size = 4  # num parallel sequences to process
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_interval = 300
eval_iters = 200
max_iters = 5000
n_heads = 4
n_block_layers = 12

## Prepare training data

Split data into training and validation set

In [8]:
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data))
print(len(val_data))

693639
77072


In [9]:
# X is training inputs
# Y is target outputs (labels)
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{context.tolist()} -> {target}")

[3] -> 55
[3, 55] -> 50
[3, 55, 50] -> 62
[3, 55, 50, 62] -> 75
[3, 55, 50, 62, 75] -> 76
[3, 55, 50, 62, 75, 76] -> 62
[3, 55, 50, 62, 75, 76, 62] -> 56
[3, 55, 50, 62, 75, 76, 62, 56] -> 0


## Set up batch function

In [10]:
batch_ixs = torch.randint(len(data) - block_size, (4,))
x = torch.stack([data[i:i+block_size] for i in batch_ixs])
print(f"{x.shape=}")

x.shape=torch.Size([4, 8])


In [11]:
# Introduce a batch dimension

torch.manual_seed(11235)
batch_size = 4 # num of independent sequences to process in each forward and backward pass of the Transformer (in parallel)
# block_size = 8 # max context length to make predictions

def get_batch(is_training =True):
    # row length = block_size
    # number of rows (height) = batch_size
    # generate a small batch of data of inputs `x` and targets `y`
    batch_data = train_data if is_training else val_data

    # e.g. ix=tensor([ 76049, 234249, 934904, 560986])
    ix = torch.randint(len(batch_data) - block_size, (batch_size,))

    # add
    x = torch.stack([batch_data[i:i+block_size] for i in ix])
    y = torch.stack([batch_data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch(is_training=True)
print('# INPUT BATCH [X]')
print(xb.shape)
print(xb)
print('# TARGET BATCH [Y]')
print(yb.shape)
print(yb)

for b in range(batch_size): # batch dim
    for t in range(block_size): # time dim
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"{context.tolist()} -> {target}")

# INPUT BATCH [X]
torch.Size([4, 8])
tensor([[ 1, 77, 65, 62,  1, 73, 72, 68],
        [63, 62,  1, 58, 71, 61,  1, 37],
        [83,  1, 60, 58, 75,  1, 71, 72],
        [ 1, 37,  1, 61, 72, 71,  7, 77]])
# TARGET BATCH [Y]
torch.Size([4, 8])
tensor([[77, 65, 62,  1, 73, 72, 68, 62],
        [62,  1, 58, 71, 61,  1, 37,  7],
        [ 1, 60, 58, 75,  1, 71, 72, 77],
        [37,  1, 61, 72, 71,  7, 77,  1]])
[1] -> 77
[1, 77] -> 65
[1, 77, 65] -> 62
[1, 77, 65, 62] -> 1
[1, 77, 65, 62, 1] -> 73
[1, 77, 65, 62, 1, 73] -> 72
[1, 77, 65, 62, 1, 73, 72] -> 68
[1, 77, 65, 62, 1, 73, 72, 68] -> 62
[63] -> 62
[63, 62] -> 1
[63, 62, 1] -> 58
[63, 62, 1, 58] -> 71
[63, 62, 1, 58, 71] -> 61
[63, 62, 1, 58, 71, 61] -> 1
[63, 62, 1, 58, 71, 61, 1] -> 37
[63, 62, 1, 58, 71, 61, 1, 37] -> 7
[83] -> 1
[83, 1] -> 60
[83, 1, 60] -> 58
[83, 1, 60, 58] -> 75
[83, 1, 60, 58, 75] -> 1
[83, 1, 60, 58, 75, 1] -> 71
[83, 1, 60, 58, 75, 1, 71] -> 72
[83, 1, 60, 58, 75, 1, 71, 72] -> 77
[1] -> 37
[1, 37] -> 1


## Define the model

Scaled dot-product attention (dry run)

In [12]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head self-attention
head_size = 16 # number of features (H)
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# Every node (token) emits two vectors:
# 1. query (Q) - what a token is looking for
# 2. key (k) - what a token is
q = query(x) # (B, T, H)
k = key(x)   # (B, T, H)

# Transpose the last two dimensions of k -> (B, H, T)
# (B, T, H) @ (B, H, T) -> (B, T, T)
ws = q @ k.transpose(-2, -1)

# only pay attention to preceding tokens
tril = torch.tril(torch.ones(T, T)) # (T, T)
# zeroes mask (T, T) broadcast through weights (B, T, T)
ws = ws.masked_fill(tril == 0, float('-inf')) # (B, T, T)
# normalise
ws = F.softmax(ws, dim=-1) # (B, T, T)
print(f"softmaxed: {ws=}")

# the content that a token has to offer
v = value(x) # (B, T, H)

# weighted aggregation of past tokens
out = ws @ v # (B, T, H)

print(f"{v=}")
print(f"{out[0]=}")

# attention weights of tokens in first sequence
# ws[0] # (T, T)

softmaxed: ws=tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [9.3721e-01, 6.2791e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [7.4353e-01, 2.2961e-01, 2.6857e-02, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.0209e-01, 2.5084e-02, 7.7802e-01, 9.4803e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.3239e-02, 8.3880e-02, 4.5793e-02, 9.4440e-02, 7.6265e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.5288e-02, 7.0410e-02, 7.1616e-01, 5.0822e-02, 1.0343e-01,
          3.3890e-02, 0.0000e+00, 0.0000e+00],
         [2.3808e-01, 9.2292e-02, 1.5704e-01, 1.8009e-01, 1.6292e-02,
          1.3875e-01, 1.7745e-01, 0.0000e+00],
         [9.7294e-01, 3.4147e-03, 1.7623e-03, 7.9526e-03, 5.9035e-03,
          4.0651e-03, 1.7310e-03, 2.2285e-03]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.

Single head of self attention

In [13]:
class Head(nn.Module):
    """A head of self attention"""

    def __init__(self, head_size: int):
        super().__init__()
        # Initialise Q, K, V
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # the mask doesn't get updated during backpropagation (optimizer step)
        # but is part of the state of the model, so saved and loaded with the model
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x: torch.Tensor):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x)   # (B, T, C)

        # compute attention scores ("affinities")
        ws = q @ k.transpose(-2, -1) # (B, T, C) @ (B, C, T) -> (B, T, T)
        ws = ws.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)

        # compute relative predictions
        ws = F.softmax(ws, dim=-1) # (B, T, T)

        v = self.value(x) # (B, T, C)
        out = ws @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


Multi-head self-attention

In [14]:
class MultiHead(nn.Module):
    """
    Multiple heads of self-attention concatenated.
    Parallelized over the channel (C) dimension.
    """

    def __init__(self, num_heads: int, head_size: int):
        super().__init__()

        # initialise multiple self-attention heads each with `head_size` dimension
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

        # projection layer to transform concatenated outputs of all heads
        # back to the original embedding dimension
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        # concat outputs of each attention head
        out = torch.cat([h(x) for h in self.heads], dim=-1)

        # project outputs to input (embed) dimensions
        out = self.proj(out)
        return out

Feed forward

In [15]:
class FeedForward(nn.Module):
    """
    Simple feed-forward NN with a single hidden layer
    [Linear -> ReLU]
    """

    def __init__(self, n_embd: int):
        super().__init__()
        # scaling inner layer channel size by 4 is a hyperparameter recommendation in
        # 3.3 Position-wise feed forward networks from Attention is All You Need
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


Transformer block

In [ ]:
class TransformerBlock(nn.Module):
    """
    Carries out self attention (communication) and feed forward (computation) on input embeddings
    """

    def __init__(self, n_embd: int, n_heads: int):
        """
        :param n_embd: embedding dimension
        :param n_heads: number of heads
        """
        super().__init__()

        # self-attention heads
        # the same 32 embedding dimensions are split across 4 heads
        head_size: int = n_embd // n_heads
        self.sa = MultiHead(n_heads, head_size)

        # feed forward layer
        self.ffwd = FeedForward(n_embd)

        # both LayerNorm layers
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor):
        """
        Applies a round of self-attention to embeddings.
        :param x: the current embeddings with shape (B, T, C)
        :return: output data following self-attention
        """

        # apply self attention to normalised plus original x for residual connection
        # using pre-norm formulation here (modern deviation from Attention is All You Need)
        norm_x = self.ln1(x)
        x = x + self.sa(norm_x)

        # apply feed forward plus original x for residual connection
        norm_x = self.ln2(x)
        x = x + self.ffwd(norm_x)
        return x


Model

In [ ]:
# number of embedding dimensions
n_embd = 32

class DecoderLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*(TransformerBlock(n_embd, n_heads=n_heads) for _ in range (n_block_layers)))
        self.ln_f = nn.LayerNorm(n_embd), # last layer normalisation

        # language-modelling head
        # maps output of transformer block (self-attention and feed-forward layers) to the vocabulary
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idxs: torch.Tensor, targets=None):

        B, T = idxs.shape

        # Sum text and position information
        tok_emb = self.token_embedding_table(idxs)                              # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)

        # sum token embeddings with broadcast position embeddings, right-aligned as (1, T, C)
        x = tok_emb + pos_emb

        x = self.blocks(x)

        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            # inference mode -> we don't need to calculate loss
            loss = None
        else:
            # training mode -> we need to calculate loss

            # negative log likelihood loss
            # pytorch wants B x T x C
            B, T, C = logits.shape
            # print(f"B, T, C = {logits.shape=}")
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) # or .view(-1)

            # Compute loss between logit and target tensors
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idxs, max_new_tokens):
        """
        Inference.
        Turns a (B, T) array of indices into (B, T+1), (B, T+2)... (B, T+N)
        :param idx: a (B, T) array of indices in the current context
        :param max_new_tokens:
        :return: a (B, T+max_new_tokens) array of indices
        """
        for _ in range(max_new_tokens):
            # crop tokens that fit into context
            cropped_idxs = idxs[:, -block_size:]
            # get predictions
            logits, loss = self(cropped_idxs)
            # take the logits for the last token in the current sequence
            logits = logits[:, -1, :] # becomes (B, C), taking the last in the T dimension
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            next_ix = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idxs = torch.cat((idxs, next_ix), dim=1) # (B, T+1)
        return idxs


def test_model():
    m = DecoderLanguageModel()
    logits, loss = m(xb, yb)
    print(f"logits: {logits.shape}")
    print(loss.item())

    context = torch.zeros([1, 1], dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))


test_model()

## Train the model

Prepare to estimate loss

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            X, Y = get_batch(is_training=split == 'train')
            logits, loss = m(X, Y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

Define the training

In [ ]:
def train(model: nn.Module, train_steps=10_000):
    loss_values = []
    loss = None
    for step in range(train_steps):

        # Sample a batch of data for training. `xb` contains the input sequences and `yb` contains the target sequences.
        xb, yb = get_batch(is_training=True)

        # Forward pass: Compute predictions (logits) and loss for this batch using the model.
        # loss= L(yb, logits) where L is loss function (cross-entropy)
        logits, loss = model(xb, yb)

        # Before computing the gradients, we need to zero out the gradients from the previous step.
        # This is because PyTorch accumulates gradients on subsequent backward passes.
        # Set gradients of all params to zero: ∇params = 0
        optimizer.zero_grad(set_to_none=True)

        # Backward pass: Compute the gradients of the loss with respect to the model's parameters.
        # ∇params = d(loss)/d(params)
        loss.backward()

        # Update the model's parameters using the computed gradients.
        # params' = params - learning_rate * ∇params
        optimizer.step()

        # store the detached loss value for this step
        loss_values.append(loss.detach().item())

        # print loss at intervals
        if step % 500 == 0:
            estimated_loss = estimate_loss()
            print(f"step {step} – training loss: {estimated_loss['train']} | val loss: {estimated_loss['val']}")

    # print loss for the last batch
    # not necessarily the average or total loss for all batches.
    if loss is not None:
        print(f"loss after {train_steps} training steps: {loss.detach().item()}")
    return loss_values

In [ ]:
model = DecoderLanguageModel()
m = model.to(device)

# Initialize the optimiser with the parameters of the model `m` and a learning rate of 1e-3.
# AdamW is a variant of the Adam optimiser that has weight decay regularization.
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # learning rate of 1e-4 for bigger NNs

train_steps = 10_000
loss_values = train(model=m, train_steps=train_steps)

### Analyse loss

In [ ]:
def smooth(values, beta):
    smoothed = []
    for v in values:
        if smoothed:
            previous = smoothed[-1]
            smoothed.append(previous * beta + (1 - beta) * v)
        else:
            smoothed.append(v)
    return smoothed

In [ ]:
import matplotlib.pyplot as plt

plot_colour = '#1f77b4'

# plot loss values
plt.figure(figsize=(10, 5))
plt.plot(loss_values, label=f"{train_steps} steps", color=plot_colour, alpha=.2)
plt.plot(smooth(loss_values, 0.995), label=f"{train_steps} steps", color=plot_colour)
plt.xlabel('training step')
plt.ylabel('loss')
plt.title('loss vs training steps')
plt.legend()
plt.show()

## Predict

In [ ]:
torch.tensor(encode("I'm from the "), dtype=torch.long)

In [ ]:
torch.zeros([1, 1], dtype=torch.long)

In [ ]:
idx = torch.tensor([encode("You used to call me on my ")], dtype=torch.long) # we're going to need more Drake albums
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))

In [51]:
torch.tensor(encode("I'm from the "), dtype=torch.long)

tensor([37,  7, 70,  1, 63, 75, 72, 70,  1, 77, 65, 62,  1])

In [42]:
torch.zeros([1, 1], dtype=torch.long)

tensor([[0]])

In [74]:
idx = torch.tensor([encode("You used to call me on my ")], dtype=torch.long) # we're going to need more Drake albums
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))

You used to call me on my lone
E'It goinf
Girls
Sound
And the slike 11: Drake]
But anyy

[Putro: Drake you Godeach
With neds poplies whit
, yeah, it's vechack's)
Nowe]
Appich, Homes me all never becone, svilly hobuld it's when I'm felus?
They Bare

[Verse 1: Eem the giivice, you""

[Ponerst"
I'm niggre it I onees be uppink
B


## Appendix

Model parameters

In [ ]:
# for param in m.parameters():
#     print(f"{type(param)=} - {param=}: {param.size()=}")

Expected loss (without training)

In [ ]:
import math
print(f"{-math.log(1/len(chars))}")